# Create SGD DB
> Run in project root directory

In [1]:
%load_ext autoreload
%autoreload 2

## Load

In [3]:
from sgd.utils import load_schemas, load_dialogs

schemas = load_schemas()
dialogs = load_dialogs()

len(schemas), len(dialogs)

(45, 22825)

## Collect DB Records

In [4]:
from collections import defaultdict

tables = defaultdict(list)

for dialog in dialogs.values():
    for turn in dialog['turns']:
        if turn['speaker'] != 'SYSTEM':
            continue
        for frame in turn['frames']:
            if 'service_results' not in frame:
                continue
            service_name = frame['service']
            intent_name = frame['service_call']['method']
            schema = schemas[service_name]
            intents_dict = {intent['name']: intent for intent in schema['intents']}
            intent = intents_dict[intent_name]
            if intent['is_transactional']:
                continue
            for result in frame['service_results']:
                tables[frame['service']].append(result)

# Deduplication
for name, table in tables.items():
    d = {str(a): a for a in table}
    tables[name] = list(d.values())


len(tables['Travel_1'])

1254

## Field Type
- field_data_type
- field_info_type

In [5]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

field_data_type = defaultdict(dict)  # service -> field -> type
for service_name, schema in schemas.items():
    table = tables[service_name]
    for slot in schema['slots']:
        field = slot['name']
        if all(item[field].isdigit() for item in table if field in item):
            field_data_type[service_name][field] = 'INTEGER'
        elif all(is_float(item[field]) for item in table if field in item):
            field_data_type[service_name][field] = 'REAL'
        else:
            field_data_type[service_name][field] = 'TEXT'

field_data_type['Travel_1']

{'location': 'TEXT',
 'attraction_name': 'TEXT',
 'category': 'TEXT',
 'phone_number': 'TEXT',
 'free_entry': 'TEXT',
 'good_for_kids': 'TEXT'}

In [8]:
field_info_type = defaultdict(dict)

for service_name, schema in schemas.items():
    query_slots = set()
    trans_slots = set()
    for intent in schema['intents']:
        if not intent['is_transactional']:
            query_slots.update(intent['required_slots'])
            query_slots.update(intent['optional_slots'].keys())
            query_slots.update(intent['result_slots'])
        else:
            trans_slots.update(intent['required_slots'])
            trans_slots.update(intent['optional_slots'].keys())
            trans_slots.update(intent['result_slots'])
    for slot in query_slots:
        trans_slots.discard(slot)

    for slot in schema['slots']:
        field = slot['name']
        if field not in trans_slots:
            field_info_type[service_name][field] = 'info'
        else:
            field_info_type[service_name][field] = 'tran'


field_info_type['Events_1']

{'category': 'info',
 'subcategory': 'info',
 'event_name': 'info',
 'date': 'info',
 'time': 'info',
 'number_of_seats': 'tran',
 'city_of_event': 'info',
 'event_location': 'info',
 'address_of_location': 'info'}

## Create Info DB

In [9]:
import sqlite3

from sgd.utils import INFO_DB_PATH

conn = sqlite3.connect(INFO_DB_PATH)
for service_name, schema in schemas.items():
    # Create Table
    sql = []
    sql.append(f'CREATE TABLE {service_name} (')
    sql.append('id INTEGER PRIMARY KEY,')
    for slot in schema['slots']:
        field = slot['name']
        if field_info_type[service_name][field] != 'info':
            continue
        type = field_data_type[service_name][field]
        sql.append(f'"{slot["name"]}" {type} COLLATE NOCASE,')
    sql[-1] = sql[-1].rstrip(',') + ')'
    sql = '\n'.join(sql)

    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()

    # Insert
    def apply_type(item, type_map):
        values = []
        for field, v in item.items():
            type = type_map[field]
            if type == 'INTEGER':
                v = int(v)
            elif type == 'REAL':
                v = float(v)
            values.append(v)
        return values

    cursor = conn.cursor()
    for item in tables[service_name]:
        sql = f'INSERT INTO {service_name} ('
        sql += ', '.join(f'"{field}"' for field in item.keys())
        sql += ') VALUES ('
        sql += ', '.join(['?'] * len(item))
        sql += ')'
        values = apply_type(item, field_data_type[service_name])
        cursor.execute(sql, values)
    conn.commit()
    cursor.close()
conn.close()

## Create Transaction DB

In [10]:
import sqlite3

from sgd.utils import TRANS_DB_PATH

conn = sqlite3.connect(TRANS_DB_PATH)
for service_name, schema in schemas.items():
    sql = f'CREATE TABLE {service_name}_Transaction ('
    sql += 'id INTEGER PRIMARY KEY,'
    for slot in schema['slots']:
        field = slot['name']
        type = field_data_type[service_name][field]
        sql += f'"{field}" {type} COLLATE NOCASE,'
    sql += 'refer_number TEXT COLLATE NOCASE)'

    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()
conn.close()